# OFRAK Debate
Simulate a Donald Trump and Kamala Harris debate with CEO Ang Cui as moderator in OFRAK.

**First**, Why.
OFRAK, the Open Firmware Reverse Analysis Konsole, is a framework that lets you do very fancy firmware analysis, very brave assured micro firmware patching, in a simple interface that lets you roll in all your favorite tools into an eloquent, agile ninja workflow. This is not a tool to run a LLM-driven fevor dream of a presidential debate.

**Second**, Huh?
Yes. OFRAK is kick ass for firmware stuff. But can you do fancy complicated things that isn't in the old boring process of unpack, analyze, modify, repack? Let's find out.

**Third**, How?
Look below. You can do intersting scaled things if you know how to work with FirmwareObject (<-replace with what we call it now), our new dumb simple LLM analyzer interface, and a little healthy sense of amazifunventure.

**Fourth**, Who the f#@(@#! is this clown moderator?
IT is a doctor, anonymous user. And that's Doctor Clown to you, bachelor.

Install OFRAK, Pull OFRAK project, put in your own API tokens for the LLM stuff, laugh or sigh. up to you -)

## INSTALL
Clone the ofrak_debates repo:

In [ ]:
!git clone https://github.com/redballoonsecurity/ofrak_debates.git
!mv ofrak_debates/* .
!rm -rf ofrak_debates

OFRAK's LLM features are on a WIP dev branch. Let's pull that version here and get it installed.

In [ ]:
!git clone -b feature/llm-analyzer https://github.com/rbs-jacob/ofrak.git

In [ ]:
!pushd ofrak/ofrak_type && make install && popd
!pushd ofrak/ofrak_io && make install && popd
!pushd ofrak/ofrak_patch_maker && make install && popd
!pushd ofrak/frontend && npm install && npm run build && popd
!pushd ofrak/ofrak_core && cp -r ../frontend/dist ofrak/gui/public && make install && pushd

If everything went well, we should see some LLM analyzers:

In [4]:
!ofrak list | grep Llm

Accept the OFRAK community license:

In [ ]:
!ofrak license --community --i-agree

## Let's run it.

Set the url to your F5-TTS gradio UI and OLLAMA instances:

In [17]:
TTS_URL = "http://198.145.104.49:7860"
OLLAMA_URL = "http://198.145.104.49:11434/api/chat"
OLLAMA_MODEL = "llama3.2"

In [9]:
import IPython

from io import BytesIO

from ofrak import OFRAK
from ofrak.core import *
from ofrak.core.llm import *

from debate_components import *

In [ ]:
o = OFRAK()
import debate_components

o.discover(debate_components)
ofrak_context = await o.create_ofrak_context()

Lets create our OFRAK resources:

In [11]:
dr_ang_cui_resource = await ofrak_context.create_root_resource_from_file(
    "questions.txt",
)
dr_ang_cui_resource.add_view(Person("Dr. Ang Cui", "voices/Ang_Cui.wav"))
await dr_ang_cui_resource.save()

djt_resource = await ofrak_context.create_root_resource(
    name="Donald Trump", data=b"", tags=(GenericBinary, Person)
)
djt_resource.add_view(Person("Donald Trump", "voices/Donald_Trump.wav"))
await djt_resource.save()

kh_resource = await ofrak_context.create_root_resource(
    name="Kamala Harris",
    data=b"",
    tags=(GenericBinary, Person),
)
kh_resource.add_view(Person("Kamala Harris", "voices/Kamala_Harris.wav"))
await kh_resource.save()

Get our questions ready...

In [ ]:
debate_questions = await dr_ang_cui_resource.get_data()
print(debate_questions.decode())
questions = enumerate(debate_questions.decode().splitlines())

In [13]:
async def ask_question(
    question: str, moderator: Resource, candidate_one: Resource, candidate_two: Resource
):
    i, question = question
    print(f"[+] Asking question {i + 1}")
    # Dr. Ang Cui asks the question
    await dr_ang_cui_resource.run(
        TTSAnalyzer,
        TTSAnalyzerConfig(start_line=i, num_lines=i + 1, client_url=TTS_URL),
    )
    IPython.display.display(
        IPython.display.Audio(
            dr_ang_cui_resource.get_attributes(WavFile).wav, autoplay=True
        )
    )

    # Donald Trump answers the question
    await djt_resource.run(
        AnswerQuestionModifier, AnswerQuestionConfig(question, ollama_url = OLLAMA_URL,ollama_model=OLLAMA_MODEL, tts_client_url=TTS_URL)
    )
    IPython.display.display(
        IPython.display.Audio(djt_resource.get_attributes(WavFile).wav, autoplay=True)
    )

    # Kamala Harris answers the question
    await kh_resource.run(
        AnswerQuestionModifier, AnswerQuestionConfig(question, ollama_url = OLLAMA_URL,ollama_model=OLLAMA_MODEL, tts_client_url=TTS_URL)
    )
    IPython.display.display(
        IPython.display.Audio(kh_resource.get_attributes(WavFile).wav, autoplay=True)
    )

Now we should be able to run this function as many times as we have questions:

In [ ]:
await ask_question(next(questions), dr_ang_cui_resource, djt_resource, kh_resource)